In [24]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
import os
from dotenv import load_dotenv
from typing import Optional
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
load_dotenv()
import json
import requests
import pandas as pd

# Extracting information using form recognizer


In [97]:
def analyze_invoice_document(file_path):
    """
    Analyzes the given invoice document using Azure Form Recognizer service.

    Parameters:
    file_path (str): Path to the invoice document file.

    Returns:
    list: A list containing the raw content extracted from the document.
    """

    # Get Azure Form Recognizer endpoint and key from environment variables
    endpoint = os.environ["FORM_RECOGNIZER_ENDPOINT"]
    key = os.environ["FORM_RECOGNIZER_KEY"]

    # Create Azure Form Recognizer credential and client
    credential = AzureKeyCredential(key)
    client = DocumentAnalysisClient(endpoint, credential)

    # Initialize a list to store raw content
    raw_content_lst = []
    raw_content = ""

    # Submit the document for analysis
    with open(file_path, "rb") as f:
        poller = client.begin_analyze_document("prebuilt-layout", document=f)

    # Wait for analysis to complete
    result = poller.result()

    # Extract raw content from the analyzed document
    for page in result.pages:
        for line in page.lines:
            raw_content += line.content + "\n"
            raw_content_lst.append(raw_content)

    # Return the list of raw content extracted from the document
    return raw_content


# using LLM to extract information from invoices 

In [427]:
sample_json_format = "{ \"line_items\":\"\",\n }"

In [429]:
json_schema_1 = {
    "title": "id",
    "description": "Extract information from invoice",
    "type": "object",
    "properties": {
        "id": {"title": "ID", "description": "assign a UUID as value to id key", "type": "string"},
        "name_of_the_company": {"title": "name_of_the_company", "description": "Identify the name of the company", "type": "string"},
        "address": {"title": "address", "description": "Identify the address of the company", "type": "string"},
        "issued_to": {"title": "issued_to", "description": "which company was the invoice issued to", "type": "string"},
        "total_amount": {"title": "total_amount", "description": "What is the total amount of the invoice", "type": "string"},
        "balance": {"title": "balance", "description": "What is the balance amount of the invoice", "type": "string"},
    },
    "required": ["id", "name_of_the_company","address","issued_to","total_amount","balance","invoice_number","invoice_date","shipment_details","line_items","payment_terms"],
}

In [430]:
json_schema_2 = {
    "title": "id",
    "description": "Extract information from invoice",
    "type": "object",
    "properties": {
        "invoice_number": {"title": "invoice_number", "description": "what is the invoice number", "type": "string"},
        "invoice_date": {"title": "invoice_date", "description": "what is the date the invoice issued", "type": "string"},
        "shipment_details": {"title": "shipment_details", "description": "Please extract all the shipment details", "type": "string"},
        "line_items": {"title": "line_items", "description": "what are line items mentioned and please follow the json pattern of {sample_json_format} format", "type": "string"},
        "payment_terms": {"title": "payment_terms", "description": "please mention only the payment terms mentioned in the invoice", "type": "string"}
    },
    "required": ["shipment_details","payment_terms", "line_items"],
}

In [431]:
def invoice_processing(json_schema, input_text):
    """
    Extracts structured information from the input text using a given JSON schema.

    Parameters:
    json_schema (dict): JSON schema describing the expected structure of the input data.
    input_text (str): Input text containing unstructured information to be processed.

    Returns:
    str: Extracted information formatted according to the JSON schema.
    """

    # Define a prompt to instruct the algorithm about its task
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a world-class algorithm for extracting information in structured formats."),
            ("human", "Use the given format to extract information from the following input: {input_text}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ]
    )

    # Initialize the language model (OpenAI GPT-3.5 Turbo) for conversation
    llm = ChatOpenAI(engine="gpt_3_5_turbo", temperature=0)

    # Create a structured output processing chain
    chain = create_structured_output_chain(json_schema, llm, prompt, verbose=True)

    # Process the input text and extract structured information
    output = chain.run(input_text)

    # Return the extracted structured information
    return output


In [432]:
def process_multiple_schemas(input_text, json_schema_1, json_schema_2):
    """
    Extracts structured information from the input text using two different JSON schemas.

    Parameters:
    input_text (str): Input text containing unstructured information to be processed.
    json_schema_1 (dict): First JSON schema describing the expected structure of the input data.
    json_schema_2 (dict): Second JSON schema describing an alternative structure of the input data.

    Returns:
    list: A list containing the extracted structured information based on both schemas.
    """

    # Initialize an empty list to store the results from different schemas
    results = []
    
    # Process input using the first schema and append the result to the list
    result_1 = invoice_processing(json_schema_1, input_text)
    results.append(result_1)
    
    # Process input using the second schema and append the result to the list
    result_2 = invoice_processing(json_schema_2, input_text)
    results.append(result_2)
    
    # Return the list of extracted structured information based on both schemas
    return results


# Dynamic Evaluation method

In [440]:
# dummy data for testing 
synthetic_data = {'id': '123456',
 'name_of_the_company': 'crayon Pty Ltd',
 'address': '1788 dandenong road clayton-3168',
 'issued_to': 'Maximum life PVT LTD',
 'total_amount': '$1000',
 'balance': '$1000',
 'invoice_number': '8540',
 'invoice_date': '31/09/2022',
 'shipment_details': '',
 'payment_terms': 'Net 40'}

In [441]:
def format_json_output(schema, data):
    """
    Formats the given data based on the provided JSON schema.

    Parameters:
    schema (dict): JSON schema describing the expected structure of the data.
    data (dict): Data to be formatted based on the schema.

    Returns:
    list: A list of dictionaries containing formatted data with titles, descriptions, and corresponding answers.
    """

    # Initialize an empty list to store the formatted output
    result = []

    # Iterate through the properties defined in the schema
    for key, value in schema["properties"].items():
        # Check if the property is an array
        if value["type"] == "array":
            # Get the nested data for the array property
            nested_data = data.get(key, [])
            # Get the schema for items in the array
            nested_schema = value["items"]
            # Process each item in the array recursively
            for item in nested_data:
                nested_result = format_json_output(nested_schema, item)
                result.extend(nested_result)
        else:
            # Extract title, description, and answer for non-array properties
            title = value.get("title")
            description = value.get("description")
            answer = data.get(key, "Not Provided")
            # Store the formatted data in a dictionary and append to the result list
            result.append({'title': title, 'description': description, 'answer': answer})

    # Return the formatted output as a list of dictionaries
    return result


In [443]:
template_1 = '''
System: You are an AI assistant. You will be given the definition of an evaluation metric for assessing the quality of an answer in a question-answering task. Your job is to compute an accurate evaluation score using the provided evaluation metric.

User:
Relevance measures how well the answer addresses the main aspects of the question, based on the context. Consider whether all and only the important aspects are contained in the answer when evaluating relevance. Given the context and question, score the relevance of the answer between zero to one using the following rating scale:
Zero stars : the answer is wrong or incomplete
One stars : the answer is completely correct

This rating value should always be an integer between 0 and 1. So the rating produced should be 0 or 1

Example 1:

context: TAX INVOICE\nIntegrated Living\nPO BOX 2567\nDANGAR NSW 2309\nAUSTRALIA\nInvoice Date\n1 Aug 2022\nInvoice Number\nINV-0716\nReference\nJanet Sissions\nABN\n41 539 697 361\nInside Out Property And\nGarden Maintenance\n65 Kula Rd\nMEDOWIE NSW 2318\nAUSTRALIA\nItem\nDescription\nQuantity\nUnit Price\nGST\nAmount AUD\nCASE\nCase Manager\nSusan Swanson\n1.00\n0.00\n0.00\nCLIENT\nClient Name and Address\nJanet Sissions\n11/287-289 Charlestown RD\nCharlestown\n1.00\n0.00\n0.00\nMOULD\nMould Removal\nAttend clients home remove on main\nbedroom lounge room and spare bed room.\nJob completed 29.07.2022\n1.00\n120.00\n10%\n120.00\nSubtotal\n120.00\nTOTAL GST 10%\n12.00\nTOTAL AUD\n132.00\nDue Date: 15 Aug 2022\nInvoices payable within 14 days to the following bank account;\nInside Out Property And Garden Maintenance\nBSB 650 000\nACC 525 656 002\nInside Out\nPROPERTY &\nGARDEN\nMaintenance\n-X ---\nPAYMENT ADVICE\nTo:\nInside Out Property And Garden Maintenance\n65 Kula Rd\nMEDOWIE NSW 2318\nAUSTRALIA\nCustomer\nIntegrated Living\nInvoice Number\nINV-0716\nAmount Due\n132.00\nDue Date\n15 Aug 2022\nAmount Enclosed\nEnter the amount you are paying above\n

question: Identify the name of the company
answer: Moonlighting Connects Pty Ltd

stars : 1

Example 2:

context: TAX INVOICE\nIntegrated Living\nPO BOX 2567\nDANGAR NSW 2309\nAUSTRALIA\nInvoice Date\n1 Aug 2022\nInvoice Number\nINV-0716\nReference\nJanet Sissions\nABN\n41 539 697 361\nInside Out Property And\nGarden Maintenance\n65 Kula Rd\nMEDOWIE NSW 2318\nAUSTRALIA\nItem\nDescription\nQuantity\nUnit Price\nGST\nAmount AUD\nCASE\nCase Manager\nSusan Swanson\n1.00\n0.00\n0.00\nCLIENT\nClient Name and Address\nJanet Sissions\n11/287-289 Charlestown RD\nCharlestown\n1.00\n0.00\n0.00\nMOULD\nMould Removal\nAttend clients home remove on main\nbedroom lounge room and spare bed room.\nJob completed 29.07.2022\n1.00\n120.00\n10%\n120.00\nSubtotal\n120.00\nTOTAL GST 10%\n12.00\nTOTAL AUD\n132.00\nDue Date: 15 Aug 2022\nInvoices payable within 14 days to the following bank account;\nInside Out Property And Garden Maintenance\nBSB 650 000\nACC 525 656 002\nInside Out\nPROPERTY &\nGARDEN\nMaintenance\n-X ---\nPAYMENT ADVICE\nTo:\nInside Out Property And Garden Maintenance\n65 Kula Rd\nMEDOWIE NSW 2318\nAUSTRALIA\nCustomer\nIntegrated Living\nInvoice Number\nINV-0716\nAmount Due\n132.00\nDue Date\n15 Aug 2022\nAmount Enclosed\nEnter the amount you are paying above\n
question: Identify the name of the company
answer: Sunlight Connects Pty Ltd

stars : 0

Example 3:

context: TAX INVOICE\nIntegrated Living\nPO BOX 2567\nDANGAR NSW 2309\nAUSTRALIA\nInvoice Date\n1 Aug 2022\nInvoice Number\nINV-0716\nReference\nJanet Sissions\nABN\n41 539 697 361\nInside Out Property And\nGarden Maintenance\n65 Kula Rd\nMEDOWIE NSW 2318\nAUSTRALIA\nItem\nDescription\nQuantity\nUnit Price\nGST\nAmount AUD\nCASE\nCase Manager\nSusan Swanson\n1.00\n0.00\n0.00\nCLIENT\nClient Name and Address\nJanet Sissions\n11/287-289 Charlestown RD\nCharlestown\n1.00\n0.00\n0.00\nMOULD\nMould Removal\nAttend clients home remove on main\nbedroom lounge room and spare bed room.\nJob completed 29.07.2022\n1.00\n120.00\n10%\n120.00\nSubtotal\n120.00\nTOTAL GST 10%\n12.00\nTOTAL AUD\n132.00\nDue Date: 15 Aug 2022\nInvoices payable within 14 days to the following bank account;\nInside Out Property And Garden Maintenance\nBSB 650 000\nACC 525 656 002\nInside Out\nPROPERTY &\nGARDEN\nMaintenance\n-X ---\nPAYMENT ADVICE\nTo:\nInside Out Property And Garden Maintenance\n65 Kula Rd\nMEDOWIE NSW 2318\nAUSTRALIA\nCustomer\nIntegrated Living\nInvoice Number\nINV-0716\nAmount Due\n132.00\nDue Date\n15 Aug 2022\nAmount Enclosed\nEnter the amount you are paying above\n
question: what is the invoice number
answer: 8540

stars : 0

Example 4:

context: TAX INVOICE\nIntegrated Living\nPO BOX 2567\nDANGAR NSW 2309\nAUSTRALIA\nInvoice Date\n1 Aug 2022\nInvoice Number\nINV-0716\nReference\nJanet Sissions\nABN\n41 539 697 361\nInside Out Property And\nGarden Maintenance\n65 Kula Rd\nMEDOWIE NSW 2318\nAUSTRALIA\nItem\nDescription\nQuantity\nUnit Price\nGST\nAmount AUD\nCASE\nCase Manager\nSusan Swanson\n1.00\n0.00\n0.00\nCLIENT\nClient Name and Address\nJanet Sissions\n11/287-289 Charlestown RD\nCharlestown\n1.00\n0.00\n0.00\nMOULD\nMould Removal\nAttend clients home remove on main\nbedroom lounge room and spare bed room.\nJob completed 29.07.2022\n1.00\n120.00\n10%\n120.00\nSubtotal\n120.00\nTOTAL GST 10%\n12.00\nTOTAL AUD\n132.00\nDue Date: 15 Aug 2022\nInvoices payable within 14 days to the following bank account;\nInside Out Property And Garden Maintenance\nBSB 650 000\nACC 525 656 002\nInside Out\nPROPERTY &\nGARDEN\nMaintenance\n-X ---\nPAYMENT ADVICE\nTo:\nInside Out Property And Garden Maintenance\n65 Kula Rd\nMEDOWIE NSW 2318\nAUSTRALIA\nCustomer\nIntegrated Living\nInvoice Number\nINV-0716\nAmount Due\n132.00\nDue Date\n15 Aug 2022\nAmount Enclosed\nEnter the amount you are paying above\n
question: what is the invoice number
answer: 1769

stars : 1
context: {context}

Use the context to score the stars for inputs
{question}
{answer}
stars:
'''

In [486]:
def evaluation(template_1, format_json_output, raw_invoice_data):
    """
    Evaluates the correctness of extracted information by comparing it with a predefined template 
    and raw invoice data using the GPT-3.5 Turbo engine.

    Parameters:
    template_1 (str): A predefined template used for generating prompts.
    format_json_output (list): A list of dictionaries containing formatted extracted information.
    raw_invoice_data (dict): Raw invoice data used as context for the evaluation.

    Returns:
    tuple: A tuple containing two lists - the first list (scoring) contains evaluation scores,
           and the second list (wrong_answers) contains extracted answers identified as incorrect.
    """

    # Define GPT-3.5 Turbo prompt templates for generating messages
    gpt3_5_prompt = PromptTemplate(
                    template=template_1,
                    input_variables=["context", "question", "answer"]
                )
    system_message_prompt = SystemMessagePromptTemplate(prompt=gpt3_5_prompt)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

    # Initialize lists to store evaluation scores and wrong answers
    scoring = []
    wrong_answers = []
    
    # Iterate through the formatted extracted information
    for i in format_json_output:
        # Generate messages using the predefined prompts and extracted information
        messages = chat_prompt.format_prompt(context=raw_invoice_data, question=i['description'], answer=i['answer']).to_messages()
        chat = ChatOpenAI(engine="gpt_3_5_turbo", temperature=0)
        
        # Generate response using GPT-3.5 Turbo
        resp = chat(messages)

        # Evaluate the response
        if resp.content == '0':
            # If the response indicates incorrect answer, store the answer and title
            wrong_answers.append({i['title']: i['answer']})
        # Store the response content (evaluation score) in the scoring list
        scoring.append(resp.content)

    # Return the evaluation scores and wrong answers as a tuple
    return scoring, wrong_answers


# (main)Invoice Data Extraction and Evaluation

In [ ]:
def process_and_evaluate_invoices(file_path, json_schema_1, json_schema_2, template_1):
    """
    Process and evaluates invoice data from the given file path using specified JSON schemas and evaluation template.

    Parameters:
    file_path (str): Path to the invoice document file.
    json_schema_1 (dict): First JSON schema describing the expected structure of the input data.
    json_schema_2 (dict): Second JSON schema describing an alternative structure of the input data.
    template_1 (str): Predefined template for generating prompts.

    Returns:
    float: Relevance scoring based on evaluation results.
    list: List of dictionaries containing incorrect answers identified during evaluation.
    """

    # Extracting invoice data
    raw_content = analyze_invoice_document(file_path)

    # Extracting information using bifurcated schemas
    Output_bifurcated_schemas = process_multiple_schemas(raw_content, json_schema_1, json_schema_2)

    # Merge output from bifurcated schemas into a single dictionary
    final_output = {key: value for d in Output_bifurcated_schemas for key, value in d.items()}

    # Format question-answer pairs
    formatted_question_answer_pairs = format_json_output(json_schema_1, final_output) + format_json_output(json_schema_2, final_output)

    # Evaluate formatted question-answer pairs
    evaluation_results = evaluation(template_1, formatted_question_answer_pairs, raw_content)

    # Extract incorrect results and calculate relevance scoring
    Incorrect_results = evaluation_results[1]
    relevance_scoring = sum(int(number) for number in evaluation_results[0]) / len(evaluation_results[0])

    # Print relevance scoring and incorrect results
    print(relevance_scoring)
    print(Incorrect_results)

    # Return relevance scoring and incorrect results
    return relevance_scoring, Incorrect_results

# Example usage
file_path = r"C:\Users\ashbhand\Desktop\openai-invoice-demo-app-1\data\Care Payment Invoice examples .pdf"
relevance_scoring, incorrect_results = process_and_evaluate_invoices(file_path, json_schema_1, json_schema_2, template_1)


# UI backend

In [ ]:
def generate_json_data(folder_path, file_names):
    """
    Processes a list of file names within a specified folder path and generates structured JSON data.

    Parameters:
    folder_path (str): The path to the folder containing the files to be processed.
    file_names (list): A list of file names within the specified folder.

    Returns:
    list: A list of dictionaries containing file URLs and JSON schema URLs (or None if not applicable) 
          for each file in the folder.
    """

    # Initialize an empty list to store processed file data
    json_data = []

    # Iterate through the file names in the specified folder
    for file_name in file_names:
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)

        # Check if the file has a .pdf extension
        if file_name.lower().endswith('.pdf'):
            # For PDF files, create a dictionary with the file URL pointing to the invoice PDF file
            # and set the JSON schema URL to None
            json_data.append({
                "file_url": f"invoicefile/{file_name}",
                "json_schema": None
            })
        else:
            # For non-PDF files, create a dictionary with the file URL set to None
            # and construct a JSON schema URL based on the file name
            json_data.append({
                "file_url": None,
                "json_schema": f"invoice/{file_name}"
            })

    # Return the list of dictionaries containing processed file URLs and JSON schema URLs
    return json_data


# Example usage:
folder_path = '/path/to/folder'  # Replace with the path to your folder containing the files
file_names = ['invoice.pdf', 'json_schema_1.txt']

json_data = generate_json_data(folder_path, file_names)
print(json.dumps(json_data, indent=4))

In [481]:
def make_a_request(json_data):
    """
    Sends a POST request to a specified URL with JSON data as the payload.

    Parameters:
    json_data (dict): JSON data to be sent in the request payload.

    Returns:
    str: Response text if the request is successful, or an error message if the request fails.
    """

    try:
        # Define the URL you want to use for the POST request
        url = "https://invoice-openai-processing.azurewebsites.net:443/api/invoice-processing/triggers/When_a_HTTP_request_is_received/invoke?api-version=2022-05-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=OmxgHm7cjDnQEy0bPQr8_HsLFINJ9NH1jCs-niR5Ps8"

        # Convert JSON data to a string
        json_data = json.dumps(json_data)

        # Define headers for the POST request
        headers = {
            "Content-Type": "application/json",  # Set content type to JSON
            # Add authorization header if required
        }

        # Send the POST request with JSON data and headers
        response = requests.post(url, data=json_data, headers=headers)

        # Check the response status code
        if response.status_code == 200:
            # If the request is successful, return the response text
            return response.text
        else:
            # If the request fails, print an error message and the response details
            print(f"POST request to '{url}' failed")
            print("Response:", response)

    except Exception as e:
        # Handle exceptions and print an error message
        print(f"An error occurred while making a request to '{url}': {e}")

In [482]:
def execute_model(absPath):
    """
    Processes a list of file names and sends a request to a specified endpoint using the make_a_request function.

    Parameters:
    absPath (list): A list of file names including PDF and TXT files.

    Returns:
    dict: JSON response received from the request.
    """

    # Initialize an empty dictionary to store file URLs and JSON schema information
    json_data = {}

    # Iterate through the file names in the specified list
    for file_name in absPath:
        # Check if the file has a .pdf extension
        if file_name.lower().endswith('.pdf'):
            # For PDF files, set the file_url entry in the json_data dictionary
            json_data["file_url"] = file_name
        # Check if the file has a .txt extension
        elif file_name.lower().endswith('.txt'):
            # For TXT files, set the json_schema entry in the json_data dictionary
            json_data["json_schema"] = file_name

    # If no JSON schema file is found, default to using a "default" schema
    if "json_schema" not in json_data:
        json_data["json_schema"] = "default"

    # Print the constructed json_data for debugging purposes
    print(json_data)

    # Make a request using the constructed json_data and store the response
    request_response = make_a_request(json_data)

    # Parse the JSON response and return it
    return json.loads(request_response)


In [ ]:
absPath = ['invoicefile/2023-10-21_20-11-17/json_schema_1.txt', 'invoicefile/2023-10-21_20-11-17/51045167.PDF']
execute_model(absPath)

# Json output approach using excel as an example

In [91]:
template = '''
System: As your AI assistant, my purpose is to assist you in extracting information from invoices and provide the details in JSON format. Here's how it works:

Example 1:
input_invoice: {input}
output_json : {output}


input : {input_invoice}
output:
'''

In [ ]:
import pandas as pd

def process_excel_to_json(excel_file_path):
    """
    Reads data from an Excel file, processes it, and returns the data as a JSON string.

    Parameters:
    excel_file_path (str): The path to the Excel file.

    Returns:
    str: JSON string representing the processed data.
    """
    # Read data from the Excel file into a pandas DataFrame
    df = pd.read_excel(excel_file_path)

    # Process the data: drop NaN values in each row and convert to JSON
    json_str = df.apply(lambda x: [x.dropna()], axis=1).to_json(orient='index', date_format='iso')

    # Return the JSON string
    return json_str

# Example usage:
# Provide the path to your Excel file
excel_file_path = 'data\\AI water oncharging example3.xlsx'
processed_json = process_excel_to_json(excel_file_path)
print(processed_json)


In [99]:
def generate_invoice_response(template, invoice_example, processed_json, input_invoice):
    """
    Generates a chat response using the specified template and input data.

    Parameters:
    template (str): Template for formatting the chat prompt.
    invoice_example (str): Input invoice example data.
    processed_json (str): Processed JSON data.
    input_invoice (str): Input invoice information.

    Returns:
    str: Chat response generated by GPT-4 model.
    """
    gpt4_prompt = PromptTemplate(
        template=template,
        input_variables=["input", "output", "input_invoice"]
    )

    system_message_prompt = SystemMessagePromptTemplate(prompt=gpt4_prompt)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

    # Format the chat prompt using the provided input data
    messages = chat_prompt.format_prompt(input=invoice_example, output=processed_json, input_invoice=input_invoice).to_messages()

    # Create a GPT-4 chat instance
    chat = ChatOpenAI(engine="gpt-4-32k", temperature=0)

    # Generate the chat response using the formatted messages
    resp = chat(messages)

    # Return the content of the response
    return resp.content


Evaluation method 

In [130]:
def extract_attributes(json_data, parent_key=''):
    """
    Recursively extracts attributes and their corresponding answers from a JSON object.
    
    Args:
        json_data (dict): The JSON object to extract attributes from.

    Returns:
        list: A list of dictionaries, where each dictionary represents an attribute and its answer.
    """
    attributes_list = []
    if isinstance(json_data, dict):
        for key, value in json_data.items():
            new_key = f"{key}" if parent_key else key
            if isinstance(value, list):
                for item in value:
                    attributes_list.extend(extract_attributes(item, new_key))
            elif isinstance(value, dict):
                attributes_list.extend(extract_attributes(value, new_key))
            else:
                attributes_list.append({new_key: value})
    return attributes_list

In [143]:
template_1 = '''
System: You are an AI assistant. You will be given the definition of an evaluation metric for assessing the quality of an answer in a attribute-answer task. Your job is to compute an accurate evaluation score using the provided evaluation metric.

User:
Relevance measures how well the answer addresses the main aspects of the attribute, based on the context. Consider whether all and only the important aspects are contained in the answer when evaluating relevance. Given the context and question, score the relevance of the answer between zero to one using the following rating scale:
Zero stars : the answer is wrong or incomplete
One stars : the answer is completely correct

This rating value should always be an integer between 0 and 1. So the rating produced should be 0 or 1

Example 1:

context: Taswarer\nTasmanian Water and Sewerage Corporation Pty Ltd ABN 47 162 220 653\nCommunity Housing Limited Att: Accounts Payable 19-23 Prospect Street BOX HILL VIC 3128\nTax invoice summary\nBilling period 01 Jul 2023 - 30 Sep 2023\nFixed charges\n$2,813.96\nVariable usage charges\n$192.14\nGST\n$0.00\nTotal new charges (this bill)\n$3,006.10\nPrevious bill\n$2,896.31\nAdjustments\n-$2,896.31\nTotal\n$3,006.10\nYour current average use is 1.95 kL/day Your current average cost is $2.21/day\nMeter reading Your water usage for this quarter is based on a meter reading\n--- * If you plan to pay in person please do not cut\nTaswarer Payment slip\nAccount no.\n240040372\nConnection address 9 Hawkins Street LATROBE TAS\nDIRECT DEBIT\nDirect Debit\nBPAY BPAY®\nWould you like to receive all TasWater notices digitally? Scan this QR code, or visit: www.taswater.com.au/digital-billing\nPlease phone 13 6992 or visit\nwww.taswater.com.au/payment\nBiller code: 117309\nRef: 3991 2400 4037 22\nAll enquiries & emergencies 13 6992\nemail enquiries@taswater.com.au website www.taswater.com.au post GPO Box 1393 Hobart Tasmania 7001\nf\nTasWater1 Tas_Water\ntas_water\nAmount due: $3,006.10\nPlease pay by: 22 Aug 2023\nYour account details\nAccount no.\n240040372\nConnection\n9 Hawkins Street\naddress\nLATROBE TAS\nStatement no. 7716974596\nDate issued\n18/07/2023\nInstallation no. 440012300\nAverage daily usage in kilolitres 3\n2\n1\n0\nJul Oct Jan Apr Jul\nRead period ending\nAmount due: $3,006.10 Please pay by: 22 Aug 2023\nRegister with BPAY View® to view and pay this bill using internet banking Registration No: 3991 2400 4037 22 For assistance contact your financial institution.\n557785-009 000206(8571 :selected:\nTax invoice itemised Billing period 01 Jul 2023 - 30 Sep 2023\nFixed or service charges\nFull Fixed Water Charge - 20mm × 8 (01/07/23-30/09/23)\n$760.48\nFire Service Fixed Charge - 100mm (01/07/23-30/09/23)\n$594.12\nFull Fixed Sewerage Charge x 8 ETs (01/07/23-30/09/23)\n$1,459.36\nSubtotal\n$2,813.96\nVariable usage charges\nVariable Water Charge (17kL @ 1.1376/kL) (19/04/23-30/06/23)\n$19.34\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (12kL @ 1.1376/kL) (19/04/23-30/06/23)\n$13.65\nVariable Water Charge (2kL @ 1.1774/kL) (01/07/23-14/07/23)\n$2.35\nVariable Water Charge (11kL @ 1.1376/kL) (19/04/23-30/06/23)\n$12.51\nVariable Water Charge (2kL @ 1.1774/kL) (01/07/23-14/07/23)\n$2.35\nVariable Water Charge (21kL @ 1.1376/kL) (19/04/23-30/06/23)\n$23.89\nVariable Water Charge (4kL @ 1.1774/kL) (01/07/23-14/07/23)\n$4.71\nVariable Water Charge (27kL @ 1.1376/kL) (19/04/23-30/06/23)\n$30.72\nVariable Water Charge (5kL @ 1.1774/kL) (01/07/23-14/07/23)\n$5.89\nVariable Water Charge (17kL @ 1.1376/kL) (19/04/23-30/06/23)\n$19.34\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (14kL @ 1.1376/kL) (19/04/23-30/06/23)\n$15.93\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (23kL @ 1.1376/kL) (19/04/23-30/06/23)\n$26.16\nVariable Water Charge (4kL @ 1.1774/kL) (01/07/23-14/07/23)\n$4.71\nSubtotal\n$192.14\nTotal\n$3,006.10\nMeter details\nMeter reading Your water usage for this quarter is based on a meter reading\nWays to pay your bill\nAdditional information\nNeed help paying your bill? We can help. Please call us on 13 6992 and we'll help make a plan that works for you.\nEligible for a concession? If you have an eligible concession card, we can apply a discount to your bill. Call us on 13 6992 or apply online at taswater.com.au/concessions\nInterest on overdue amounts Daily interest is charged on overdue amounts from the due date at the monthly 90-day Bank Accepted Bill Rate plus 6%.\nChanged your contact details? Please call us on 13 6992 and we'll update your details or go to taswater.com.au/update\nSelling a property? If you sell your property, you are still liable for TasWater bills until your solicitor or conveyancer lets us know that ownership has changed. For more information, see taswater.com.au/ownership\nReading your meter every 3 months We need to be able to see your meter and access it easily and safely. Please keep your plants pruned around the meter, restrain your pets and leave your gate unlocked. If we can't safely read your meter, we will estimate your water usage for the next bill.\nYour information is private See our website to read our Privacy and Credit Reporting Policy at taswater.com.au/privacy\nHearing or speech impaired? Call us via the National Relay Service: TTY: call 13 3677 then ask for 13 6992. Speak and Listen: call 1300 555 727 then ask for 13 6992.\nNeed an interpreter? TIS National provides immediate phone interpreting services in 160 languages. Call 13 14 50.\nYour account details\nTaswater\nTasmanian Government\ncentrelink\nAccount\nService Tasmania\nPhone 1300 729 859 Billpay code: 8261 Reference: 39912 40040 37231\nCentrepay\nContact Centrelink for setup humanservices.gov.au/ centrepay Reference: 240040372\nBy mail Send this slip with your cheque (no staples) to: GPO Box 1393 Hobart Tas 7001\nRef\nAmount\nPost Billpay\nReceipt\nAustralia Post Pay in store at Australia\n*444 3991 240040372 31\nDate\nPost\nThanks! :selected:

Company: Tasmanian Water and Sewerage Corporation Pty Ltd

stars : 1

Example 2:

context: Taswarer\nTasmanian Water and Sewerage Corporation Pty Ltd ABN 47 162 220 653\nCommunity Housing Limited Att: Accounts Payable 19-23 Prospect Street BOX HILL VIC 3128\nTax invoice summary\nBilling period 01 Jul 2023 - 30 Sep 2023\nFixed charges\n$2,813.96\nVariable usage charges\n$192.14\nGST\n$0.00\nTotal new charges (this bill)\n$3,006.10\nPrevious bill\n$2,896.31\nAdjustments\n-$2,896.31\nTotal\n$3,006.10\nYour current average use is 1.95 kL/day Your current average cost is $2.21/day\nMeter reading Your water usage for this quarter is based on a meter reading\n--- * If you plan to pay in person please do not cut\nTaswarer Payment slip\nAccount no.\n240040372\nConnection address 9 Hawkins Street LATROBE TAS\nDIRECT DEBIT\nDirect Debit\nBPAY BPAY®\nWould you like to receive all TasWater notices digitally? Scan this QR code, or visit: www.taswater.com.au/digital-billing\nPlease phone 13 6992 or visit\nwww.taswater.com.au/payment\nBiller code: 117309\nRef: 3991 2400 4037 22\nAll enquiries & emergencies 13 6992\nemail enquiries@taswater.com.au website www.taswater.com.au post GPO Box 1393 Hobart Tasmania 7001\nf\nTasWater1 Tas_Water\ntas_water\nAmount due: $3,006.10\nPlease pay by: 22 Aug 2023\nYour account details\nAccount no.\n240040372\nConnection\n9 Hawkins Street\naddress\nLATROBE TAS\nStatement no. 7716974596\nDate issued\n18/07/2023\nInstallation no. 440012300\nAverage daily usage in kilolitres 3\n2\n1\n0\nJul Oct Jan Apr Jul\nRead period ending\nAmount due: $3,006.10 Please pay by: 22 Aug 2023\nRegister with BPAY View® to view and pay this bill using internet banking Registration No: 3991 2400 4037 22 For assistance contact your financial institution.\n557785-009 000206(8571 :selected:\nTax invoice itemised Billing period 01 Jul 2023 - 30 Sep 2023\nFixed or service charges\nFull Fixed Water Charge - 20mm × 8 (01/07/23-30/09/23)\n$760.48\nFire Service Fixed Charge - 100mm (01/07/23-30/09/23)\n$594.12\nFull Fixed Sewerage Charge x 8 ETs (01/07/23-30/09/23)\n$1,459.36\nSubtotal\n$2,813.96\nVariable usage charges\nVariable Water Charge (17kL @ 1.1376/kL) (19/04/23-30/06/23)\n$19.34\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (12kL @ 1.1376/kL) (19/04/23-30/06/23)\n$13.65\nVariable Water Charge (2kL @ 1.1774/kL) (01/07/23-14/07/23)\n$2.35\nVariable Water Charge (11kL @ 1.1376/kL) (19/04/23-30/06/23)\n$12.51\nVariable Water Charge (2kL @ 1.1774/kL) (01/07/23-14/07/23)\n$2.35\nVariable Water Charge (21kL @ 1.1376/kL) (19/04/23-30/06/23)\n$23.89\nVariable Water Charge (4kL @ 1.1774/kL) (01/07/23-14/07/23)\n$4.71\nVariable Water Charge (27kL @ 1.1376/kL) (19/04/23-30/06/23)\n$30.72\nVariable Water Charge (5kL @ 1.1774/kL) (01/07/23-14/07/23)\n$5.89\nVariable Water Charge (17kL @ 1.1376/kL) (19/04/23-30/06/23)\n$19.34\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (14kL @ 1.1376/kL) (19/04/23-30/06/23)\n$15.93\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (23kL @ 1.1376/kL) (19/04/23-30/06/23)\n$26.16\nVariable Water Charge (4kL @ 1.1774/kL) (01/07/23-14/07/23)\n$4.71\nSubtotal\n$192.14\nTotal\n$3,006.10\nMeter details\nMeter reading Your water usage for this quarter is based on a meter reading\nWays to pay your bill\nAdditional information\nNeed help paying your bill? We can help. Please call us on 13 6992 and we'll help make a plan that works for you.\nEligible for a concession? If you have an eligible concession card, we can apply a discount to your bill. Call us on 13 6992 or apply online at taswater.com.au/concessions\nInterest on overdue amounts Daily interest is charged on overdue amounts from the due date at the monthly 90-day Bank Accepted Bill Rate plus 6%.\nChanged your contact details? Please call us on 13 6992 and we'll update your details or go to taswater.com.au/update\nSelling a property? If you sell your property, you are still liable for TasWater bills until your solicitor or conveyancer lets us know that ownership has changed. For more information, see taswater.com.au/ownership\nReading your meter every 3 months We need to be able to see your meter and access it easily and safely. Please keep your plants pruned around the meter, restrain your pets and leave your gate unlocked. If we can't safely read your meter, we will estimate your water usage for the next bill.\nYour information is private See our website to read our Privacy and Credit Reporting Policy at taswater.com.au/privacy\nHearing or speech impaired? Call us via the National Relay Service: TTY: call 13 3677 then ask for 13 6992. Speak and Listen: call 1300 555 727 then ask for 13 6992.\nNeed an interpreter? TIS National provides immediate phone interpreting services in 160 languages. Call 13 14 50.\nYour account details\nTaswater\nTasmanian Government\ncentrelink\nAccount\nService Tasmania\nPhone 1300 729 859 Billpay code: 8261 Reference: 39912 40040 37231\nCentrepay\nContact Centrelink for setup humanservices.gov.au/ centrepay Reference: 240040372\nBy mail Send this slip with your cheque (no staples) to: GPO Box 1393 Hobart Tas 7001\nRef\nAmount\nPost Billpay\nReceipt\nAustralia Post Pay in store at Australia\n*444 3991 240040372 31\nDate\nPost\nThanks! :selected:
ABN : 47 152 220 656

stars : 0

Example 3:

context: Taswarer\nTasmanian Water and Sewerage Corporation Pty Ltd ABN 47 162 220 653\nCommunity Housing Limited Att: Accounts Payable 19-23 Prospect Street BOX HILL VIC 3128\nTax invoice summary\nBilling period 01 Jul 2023 - 30 Sep 2023\nFixed charges\n$2,813.96\nVariable usage charges\n$192.14\nGST\n$0.00\nTotal new charges (this bill)\n$3,006.10\nPrevious bill\n$2,896.31\nAdjustments\n-$2,896.31\nTotal\n$3,006.10\nYour current average use is 1.95 kL/day Your current average cost is $2.21/day\nMeter reading Your water usage for this quarter is based on a meter reading\n--- * If you plan to pay in person please do not cut\nTaswarer Payment slip\nAccount no.\n240040372\nConnection address 9 Hawkins Street LATROBE TAS\nDIRECT DEBIT\nDirect Debit\nBPAY BPAY®\nWould you like to receive all TasWater notices digitally? Scan this QR code, or visit: www.taswater.com.au/digital-billing\nPlease phone 13 6992 or visit\nwww.taswater.com.au/payment\nBiller code: 117309\nRef: 3991 2400 4037 22\nAll enquiries & emergencies 13 6992\nemail enquiries@taswater.com.au website www.taswater.com.au post GPO Box 1393 Hobart Tasmania 7001\nf\nTasWater1 Tas_Water\ntas_water\nAmount due: $3,006.10\nPlease pay by: 22 Aug 2023\nYour account details\nAccount no.\n240040372\nConnection\n9 Hawkins Street\naddress\nLATROBE TAS\nStatement no. 7716974596\nDate issued\n18/07/2023\nInstallation no. 440012300\nAverage daily usage in kilolitres 3\n2\n1\n0\nJul Oct Jan Apr Jul\nRead period ending\nAmount due: $3,006.10 Please pay by: 22 Aug 2023\nRegister with BPAY View® to view and pay this bill using internet banking Registration No: 3991 2400 4037 22 For assistance contact your financial institution.\n557785-009 000206(8571 :selected:\nTax invoice itemised Billing period 01 Jul 2023 - 30 Sep 2023\nFixed or service charges\nFull Fixed Water Charge - 20mm × 8 (01/07/23-30/09/23)\n$760.48\nFire Service Fixed Charge - 100mm (01/07/23-30/09/23)\n$594.12\nFull Fixed Sewerage Charge x 8 ETs (01/07/23-30/09/23)\n$1,459.36\nSubtotal\n$2,813.96\nVariable usage charges\nVariable Water Charge (17kL @ 1.1376/kL) (19/04/23-30/06/23)\n$19.34\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (12kL @ 1.1376/kL) (19/04/23-30/06/23)\n$13.65\nVariable Water Charge (2kL @ 1.1774/kL) (01/07/23-14/07/23)\n$2.35\nVariable Water Charge (11kL @ 1.1376/kL) (19/04/23-30/06/23)\n$12.51\nVariable Water Charge (2kL @ 1.1774/kL) (01/07/23-14/07/23)\n$2.35\nVariable Water Charge (21kL @ 1.1376/kL) (19/04/23-30/06/23)\n$23.89\nVariable Water Charge (4kL @ 1.1774/kL) (01/07/23-14/07/23)\n$4.71\nVariable Water Charge (27kL @ 1.1376/kL) (19/04/23-30/06/23)\n$30.72\nVariable Water Charge (5kL @ 1.1774/kL) (01/07/23-14/07/23)\n$5.89\nVariable Water Charge (17kL @ 1.1376/kL) (19/04/23-30/06/23)\n$19.34\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (14kL @ 1.1376/kL) (19/04/23-30/06/23)\n$15.93\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (23kL @ 1.1376/kL) (19/04/23-30/06/23)\n$26.16\nVariable Water Charge (4kL @ 1.1774/kL) (01/07/23-14/07/23)\n$4.71\nSubtotal\n$192.14\nTotal\n$3,006.10\nMeter details\nMeter reading Your water usage for this quarter is based on a meter reading\nWays to pay your bill\nAdditional information\nNeed help paying your bill? We can help. Please call us on 13 6992 and we'll help make a plan that works for you.\nEligible for a concession? If you have an eligible concession card, we can apply a discount to your bill. Call us on 13 6992 or apply online at taswater.com.au/concessions\nInterest on overdue amounts Daily interest is charged on overdue amounts from the due date at the monthly 90-day Bank Accepted Bill Rate plus 6%.\nChanged your contact details? Please call us on 13 6992 and we'll update your details or go to taswater.com.au/update\nSelling a property? If you sell your property, you are still liable for TasWater bills until your solicitor or conveyancer lets us know that ownership has changed. For more information, see taswater.com.au/ownership\nReading your meter every 3 months We need to be able to see your meter and access it easily and safely. Please keep your plants pruned around the meter, restrain your pets and leave your gate unlocked. If we can't safely read your meter, we will estimate your water usage for the next bill.\nYour information is private See our website to read our Privacy and Credit Reporting Policy at taswater.com.au/privacy\nHearing or speech impaired? Call us via the National Relay Service: TTY: call 13 3677 then ask for 13 6992. Speak and Listen: call 1300 555 727 then ask for 13 6992.\nNeed an interpreter? TIS National provides immediate phone interpreting services in 160 languages. Call 13 14 50.\nYour account details\nTaswater\nTasmanian Government\ncentrelink\nAccount\nService Tasmania\nPhone 1300 729 859 Billpay code: 8261 Reference: 39912 40040 37231\nCentrepay\nContact Centrelink for setup humanservices.gov.au/ centrepay Reference: 240040372\nBy mail Send this slip with your cheque (no staples) to: GPO Box 1393 Hobart Tas 7001\nRef\nAmount\nPost Billpay\nReceipt\nAustralia Post Pay in store at Australia\n*444 3991 240040372 31\nDate\nPost\nThanks! :selected:
Billing Address : 1788 dandenong road clayton-3168

stars : 0

Example 4:

context: Taswarer\nTasmanian Water and Sewerage Corporation Pty Ltd ABN 47 162 220 653\nCommunity Housing Limited Att: Accounts Payable 19-23 Prospect Street BOX HILL VIC 3128\nTax invoice summary\nBilling period 01 Jul 2023 - 30 Sep 2023\nFixed charges\n$2,813.96\nVariable usage charges\n$192.14\nGST\n$0.00\nTotal new charges (this bill)\n$3,006.10\nPrevious bill\n$2,896.31\nAdjustments\n-$2,896.31\nTotal\n$3,006.10\nYour current average use is 1.95 kL/day Your current average cost is $2.21/day\nMeter reading Your water usage for this quarter is based on a meter reading\n--- * If you plan to pay in person please do not cut\nTaswarer Payment slip\nAccount no.\n240040372\nConnection address 9 Hawkins Street LATROBE TAS\nDIRECT DEBIT\nDirect Debit\nBPAY BPAY®\nWould you like to receive all TasWater notices digitally? Scan this QR code, or visit: www.taswater.com.au/digital-billing\nPlease phone 13 6992 or visit\nwww.taswater.com.au/payment\nBiller code: 117309\nRef: 3991 2400 4037 22\nAll enquiries & emergencies 13 6992\nemail enquiries@taswater.com.au website www.taswater.com.au post GPO Box 1393 Hobart Tasmania 7001\nf\nTasWater1 Tas_Water\ntas_water\nAmount due: $3,006.10\nPlease pay by: 22 Aug 2023\nYour account details\nAccount no.\n240040372\nConnection\n9 Hawkins Street\naddress\nLATROBE TAS\nStatement no. 7716974596\nDate issued\n18/07/2023\nInstallation no. 440012300\nAverage daily usage in kilolitres 3\n2\n1\n0\nJul Oct Jan Apr Jul\nRead period ending\nAmount due: $3,006.10 Please pay by: 22 Aug 2023\nRegister with BPAY View® to view and pay this bill using internet banking Registration No: 3991 2400 4037 22 For assistance contact your financial institution.\n557785-009 000206(8571 :selected:\nTax invoice itemised Billing period 01 Jul 2023 - 30 Sep 2023\nFixed or service charges\nFull Fixed Water Charge - 20mm × 8 (01/07/23-30/09/23)\n$760.48\nFire Service Fixed Charge - 100mm (01/07/23-30/09/23)\n$594.12\nFull Fixed Sewerage Charge x 8 ETs (01/07/23-30/09/23)\n$1,459.36\nSubtotal\n$2,813.96\nVariable usage charges\nVariable Water Charge (17kL @ 1.1376/kL) (19/04/23-30/06/23)\n$19.34\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (12kL @ 1.1376/kL) (19/04/23-30/06/23)\n$13.65\nVariable Water Charge (2kL @ 1.1774/kL) (01/07/23-14/07/23)\n$2.35\nVariable Water Charge (11kL @ 1.1376/kL) (19/04/23-30/06/23)\n$12.51\nVariable Water Charge (2kL @ 1.1774/kL) (01/07/23-14/07/23)\n$2.35\nVariable Water Charge (21kL @ 1.1376/kL) (19/04/23-30/06/23)\n$23.89\nVariable Water Charge (4kL @ 1.1774/kL) (01/07/23-14/07/23)\n$4.71\nVariable Water Charge (27kL @ 1.1376/kL) (19/04/23-30/06/23)\n$30.72\nVariable Water Charge (5kL @ 1.1774/kL) (01/07/23-14/07/23)\n$5.89\nVariable Water Charge (17kL @ 1.1376/kL) (19/04/23-30/06/23)\n$19.34\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (14kL @ 1.1376/kL) (19/04/23-30/06/23)\n$15.93\nVariable Water Charge (3kL @ 1.1774/kL) (01/07/23-14/07/23)\n$3.53\nVariable Water Charge (23kL @ 1.1376/kL) (19/04/23-30/06/23)\n$26.16\nVariable Water Charge (4kL @ 1.1774/kL) (01/07/23-14/07/23)\n$4.71\nSubtotal\n$192.14\nTotal\n$3,006.10\nMeter details\nMeter reading Your water usage for this quarter is based on a meter reading\nWays to pay your bill\nAdditional information\nNeed help paying your bill? We can help. Please call us on 13 6992 and we'll help make a plan that works for you.\nEligible for a concession? If you have an eligible concession card, we can apply a discount to your bill. Call us on 13 6992 or apply online at taswater.com.au/concessions\nInterest on overdue amounts Daily interest is charged on overdue amounts from the due date at the monthly 90-day Bank Accepted Bill Rate plus 6%.\nChanged your contact details? Please call us on 13 6992 and we'll update your details or go to taswater.com.au/update\nSelling a property? If you sell your property, you are still liable for TasWater bills until your solicitor or conveyancer lets us know that ownership has changed. For more information, see taswater.com.au/ownership\nReading your meter every 3 months We need to be able to see your meter and access it easily and safely. Please keep your plants pruned around the meter, restrain your pets and leave your gate unlocked. If we can't safely read your meter, we will estimate your water usage for the next bill.\nYour information is private See our website to read our Privacy and Credit Reporting Policy at taswater.com.au/privacy\nHearing or speech impaired? Call us via the National Relay Service: TTY: call 13 3677 then ask for 13 6992. Speak and Listen: call 1300 555 727 then ask for 13 6992.\nNeed an interpreter? TIS National provides immediate phone interpreting services in 160 languages. Call 13 14 50.\nYour account details\nTaswater\nTasmanian Government\ncentrelink\nAccount\nService Tasmania\nPhone 1300 729 859 Billpay code: 8261 Reference: 39912 40040 37231\nCentrepay\nContact Centrelink for setup humanservices.gov.au/ centrepay Reference: 240040372\nBy mail Send this slip with your cheque (no staples) to: GPO Box 1393 Hobart Tas 7001\nRef\nAmount\nPost Billpay\nReceipt\nAustralia Post Pay in store at Australia\n*444 3991 240040372 31\nDate\nPost\nThanks! :selected:
Billing Address : Community Housing Limited Att : Accounts Payable 19-23 Prospect Street BOX HILL VIC 3128

stars : 1
context: {context}

Use the context to score the stars for attribute and answer just give star values
{attribute}:{answer}
stars:
'''

In [144]:
def evaluation(template_1, format_json_output, raw_invoice_data):
    """
    Evaluates the correctness of extracted information by comparing it with a predefined template 
    and raw invoice data using the GPT-3.5 Turbo engine.

    Parameters:
    template_1 (str): A predefined template used for generating prompts.
    format_json_output (list): A list of dictionaries containing formatted extracted information.
    raw_invoice_data (dict): Raw invoice data used as context for the evaluation.

    Returns:
    tuple: A tuple containing two lists - the first list (scoring) contains evaluation scores,
           and the second list (wrong_answers) contains extracted answers identified as incorrect.
    """

    # Define GPT-3.5 Turbo prompt templates for generating messages
    gpt3_5_prompt = PromptTemplate(
                    template=template_1,
                    input_variables=["context", "attribute", "answer"]
                )
    system_message_prompt = SystemMessagePromptTemplate(prompt=gpt3_5_prompt)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

    # Initialize lists to store evaluation scores and wrong answers
    scoring = []
    wrong_answers = []
    
    # Iterate through the formatted extracted information
    for dict in format_json_output:
        # Generate messages using the predefined prompts and extracted information
        for key, value in dict.items():
            messages = chat_prompt.format_prompt(context=raw_invoice_data, attribute=key, answer=value).to_messages()
            question=key
            print(question)
            chat = ChatOpenAI(engine="gpt-4-32k", temperature=0)
            
            # Generate response using GPT-3.5 Turbo
            resp = chat(messages)

            # Evaluate the response
            if resp.content == '0':
                # If the response indicates incorrect answer, store the answer and title
                wrong_answers.append({key: value})
            # Store the response content (evaluation score) in the scoring list
            scoring.append(resp.content)

    # Return the evaluation scores and wrong answers as a tuple
    return scoring, wrong_answers

(main)Generating json by giving excel example

In [ ]:
file_path = r"data\Multi-8763880 JUN23_20230615.pdf"
invoice_example = analyze_invoice_document(file_path)
file_path_1 = r"data\Multi-8763880 JUN23_20230615.pdf"
input_invoice = analyze_invoice_document(file_path_1)
response = generate_invoice_response(template, invoice_example, processed_json, input_invoice)
json_response = json.loads(response)
attributes_list = extract_attributes(json_response)
evaluation = evaluation(template_1, attributes_list, input_invoice)
relevance_scoring = sum(int(number) for number in evaluation[0])/len(evaluation[0])


# 2nd approach

In [501]:
json_schema = {
  "title": "Data Extraction Schema",
  "description": "Extract information from an invoice",
  "type": "object",
  "properties": {
    "attribute1": { "type": "string", "title": "Title for Attribute 1", "description": "Description for Attribute 1" },
    "attribute2": { "type": "string", "title": "Title for Attribute 2", "description": "Description for Attribute 2" },
    "attribute3": { "type": "string", "title": "Title for Attribute 3", "description": "Description for Attribute 3" },
    "attribute4": { "type": "string", "title": "Title for Attribute 4", "description": "Description for Attribute 4" },
    "attribute5": { "type": "string", "title": "Title for Attribute 5", "description": "Description for Attribute 5" }
  }
}


In [502]:
template = '''
System: You are an AI assistant. Please generate a JSON schema for extracting information from the given json example extract properties description by asking a question and tittle.


Example 1

json instance: {output}
jSON Schema: {json_schema}

json instance:{input}
jSON Schema:

'''


In [503]:
def generate_schema_response(template, json_str, raw_content, json_schema):
    """
    Generates a chat response using the specified template and input data.

    Parameters:
    template (str): Template for formatting the chat prompt.
    json_str (str): JSON string data.
    raw_content (str): Raw content from the document.
    json_schema (str): JSON schema information.

    Returns:
    str: Chat response generated by GPT-4 model.
    """
    gpt3_5_prompt = PromptTemplate(
        template=template,
        input_variables=['output', 'input', 'json_schema']
    )

    system_message_prompt = SystemMessagePromptTemplate(prompt=gpt3_5_prompt)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

    # Format the chat prompt using the provided input data
    messages = chat_prompt.format_prompt(output=json_str, input=raw_content, json_schema=json_schema).to_messages()

    # Create a GPT-4 chat instance
    chat = ChatOpenAI(engine="gpt-4-32k", temperature=0)

    # Generate the chat response using the formatted messages
    resp = chat(messages)

    # Return the content of the response
    return resp.content



In [ ]:
response = generate_schema_response(template, processed_json, input_invoice, json_schema)
print(response)